In [6]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import threading
import undetected_chromedriver as uc

# Create a thread-local driver pool to reuse drivers per thread.
driver_pool = {}

In [7]:
def get_driver():
    """Initialize a headless Chrome WebDriver for Selenium"""
    thread_id = threading.get_ident()
    if thread_id not in driver_pool:
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument("--disable-infobars")
        chrome_options.add_argument("--disable-extensions")
        chrome_options.add_argument("--disable-popup-blocking")
        chrome_options.add_argument("--disable-notifications")
        chrome_options.add_argument("--disable-translate")
        chrome_options.add_argument("--disable-application-cache")
        chrome_options.add_argument("--disable-background-networking")
        chrome_options.add_argument("--disable-client-side-phishing-detection")
        chrome_options.add_argument("--disable-component-update")
        chrome_options.add_argument("--disable-default-apps")
        chrome_options.add_argument("--disable-gpu")
       
 
        prefs = {"profile.managed_default_content_settings.images": 2}
        chrome_options.add_experimental_option("prefs", prefs)
        driver_pool[thread_id] = webdriver.Chrome(options=chrome_options)
    return driver_pool[thread_id]


def extract_page_info(url: str):
    """Extract page info using Selenium WebDriver (not async, so we use threads)"""
    driver = get_driver()
    try:
        driver.get(url)
        time.sleep(2)  # Allow time for page to load
        # Attempt to auto-click common cookie/consent buttons
        try:
            driver.execute_script(
                'document.querySelectorAll(\'[aria-label="Accept cookies"],'
                "button.cookie-accept, button#accept-cookies').forEach(el => el.click());"
            )
        except Exception:
            pass  # Ignore if not found
        title = driver.title
        # Grab first 200 characters of page source for a snippet
        page_snippet = driver.page_source
    except Exception as e:
        title = f"Error: {e}"
        page_snippet = ""
    return {"url": url, "title": title, "snippet": page_snippet}

In [8]:
res = extract_page_info("https://www.mid-day.com/news/india-news/article/maharashtra-pm-modi-to-visit-rss-founder-hedgewars-memorial-in-nagpur-on-march-30-23508634")

In [9]:
soup = BeautifulSoup(res["snippet"], "html.parser")

# Extract all text
text = soup.get_text(separator="\n", strip=True)
print(text)

Maharashtra: PM Modi to visit RSS founder Hedgewar’s memorial in Nagpur on March 30
Recent Searches
Trending News
Air Quality : Moderate
28 March, 2025 (Friday)
Latest Updates
Web Stories
Today's E-Paper
Mid-day Gujarati
Inquilab
Mid-day Hindi
SUBSCRIBE
×
Web Stories
E-Paper
Friday, 28 March, 2025
Mid-day Gujarati
Inquilab
Mid-day Hindi
Home
Latest News
Mumbai
Mumbai News
Mumbai Crime News
Mumbai Rains
Entertainment
Bollywood
Web Series
Television
Hollywood
Regional Cinema
Korean Entertainment News
Celebrity Life
Weddings And Relationships
Fashion And Beauty
Parenting
Diet
Leisure
Pawrenting
Activism
Well-being
Luxury
Newsmakers
Podcast
News
India News
World News
Offbeat News
Opinion
Sports
Cricket
Football
Other Sports
Lifestyle
Travel
Sex and Relationships
Food
Health & Fitness
Culture
Fashion
Nature and Wildlife
Photos
Mid-Day Web Stories
Timepass
Polls
Quiz
Crosswords
Sudoku
Manjul
Sunday Mid-Day
Mumbai Guide
Mumbai Food
Things to do
Famous Personalities
Health & Wellness
Mid-Day G